In [5]:
# -*- coding: utf-8 -*-

try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox

from sokobanXSBLevels import *

from enum import Enum

from datetime import timedelta
import json




class Direction(Enum):
    Up = 1
    Down = 2
    Left = 3
    Right = 4


class Position:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f'Position({self.x}, {self.y})'

    def getX(self):
        return self.x

    def getY(self):
        return self.y

    def setX(self, newx):
        self.x = newx

    def setY(self, newy):
        self.y = newy

    def position_suivante(self, direction, distance):
        new_x, new_y = self.x, self.y

        if direction == Direction.Up:
            new_y -= distance
        elif direction == Direction.Down:
            new_y += distance
        elif direction == Direction.Left:
            new_x -= distance
        elif direction == Direction.Right:
            new_x += distance

        return Position(new_x, new_y)

    def est_valide_dans_entrepot(self, entrepot):
        return entrepot.isPositionValid(self)

    def de_position_a_coordonné_dans_canvas(self, elem):
        coord_x = self.getX() * elem.getWidth()
        coord_y = self.getY() * elem.getHeight()
        return Position(coord_x, coord_y)




class WharehousePlan:

    def __init__(self):
        self.rawMatrix = []

    def appendRow(self, row):
        self.rawMatrix.append(row)

    def get_element(self, position):
        x, y = position.getX(), position.getY()
        if 0 <= y < len(self.rawMatrix) and 0 <= x < len(self.rawMatrix[y]):
            return self.rawMatrix[y][x]
        return None

    def place_element(self, position, elem):
        x, y = position.getX(), position.getY()
        if 0 <= y < len(self.rawMatrix) and 0 <= x < len(self.rawMatrix[y]):
            self.rawMatrix[y][x] = elem

    def isPositionValid(self, position):
        return self.get_element(position) is not None

    def hasFreePlaceAt(self, position):
        elem = self.get_element(position)
        return elem is not None and elem.isFreePlace()

    def asXsbMatrix(self):
        xsb_matrix = []

        for row in self.rawMatrix:
            xsb_row = []

            for elem in row:
                if elem:  
                    xsb_row.append(elem.asXsb())
                else:  
                    
                    xsb_row.append(' ')

            
            xsb_matrix.append(xsb_row)

        return xsb_matrix




class Floor(object):

    def __init__(self):
        None

    def isMovable(self):
        return False

    def canBeCovered(self):
        return True

    def xsbChar(self):
        return ' '

    
    def isFreePlace(self):
        return True




class Goal(object):
    def __init__(self, canvas, position):
        lien_image = 'image/goal.png'
        self.image = tk.PhotoImage(file=lien_image)
        self.width = self.image.width()
        self.height = self.image.height()
        self.position = position

        pos = self.position.de_position_a_coordonné_dans_canvas(self)
        canvas.create_image(pos.getX(), pos.getY(),
                            image=self.image, tags="static", anchor=tk.NW)

    def isMovable(self):
        return False

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width

    def canBeCovered(self):
        return True

    def xsbChar(self):
        return '.'# symbole que l'on met dans la matrice
        

    def isFreePlace(self):
        return False




class Wall(object):
    def __init__(self, canvas, position):
        lien_image = 'image/wall.png'
        self.image = tk.PhotoImage(file=lien_image)
        self.width = self.image.width()
        self.height = self.image.height()
        self.position = position

        pos = self.position.de_position_a_coordonné_dans_canvas(self)
        canvas.create_image(pos.getX(), pos.getY(),
                            image=self.image, tags="static", anchor=tk.NW)

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width

    def isMovable(self):
        return False

    def canBeCovered(self):
        return False

    def xsbChar(self):
        return '#'

    def isFreePlace(self):
        return False




class Box(object):
    def __init__(self, canvas, wharehouse, position, objectif):
        self.objectif = objectif
        if self.objectif == True:
            lien_image = 'image/boxOnTarget.png'
        else:
            lien_image = 'image/box.png'

        self.image = tk.PhotoImage(file=lien_image)
        self.width = self.image.width()
        self.height = self.image.height()
        self.position = position
        pos = self.position.de_position_a_coordonné_dans_canvas(self)
        self.image_id = canvas.create_image(
            pos.getX(), pos.getY(), image=self.image, tags="movable", anchor=tk.NW)
        self.wharehouse = wharehouse
        self.canvas = canvas
        self.animation = None
        self.circleDiameter = 50
        self.on_goal = objectif

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width

    def isMovable(self):
        return True

    def canBeCovered(self):
        return False

    def se_deplace_vers(self, direction):
        if direction == Direction.Up:
            dir_x = 0
            dir_y = -1
        elif direction == Direction.Down:
            dir_x = 0
            dir_y = 1
        elif direction == Direction.Left:
            dir_x = -1
            dir_y = 0
        elif direction == Direction.Right:
            dir_x = 1
            dir_y = 0
        else:
            dir_x = 0
            dir_y = 0

        new_x = self.position.getX() + dir_x
        new_y = self.position.getY() + dir_y

        self.canvas.delete(self.image_id)
        if self.objectif == False:
            self.image = tk.PhotoImage(file='image/box.png')
        else:
            self.image = tk.PhotoImage(file='image/boxOnTarget.png')

        self.image_id = self.canvas.create_image(
            new_x * 64, new_y * 64, image=self.image, anchor=tk.NW, tag="movable")

        self.position.setX(new_x)
        self.position.setY(new_y)

    def xsbChar(self):
        if self.under.isFreePlace():
            return '$'
        else:
            return '*'

    def isFreePlace(self):
        return False

    def objectif_couvert(self):
        self.canvas.after(100, self.objectif_couvert_animation)

    def delete_old_animation(self):
        self.canvas.delete(self.animation)
        self.animation = None
        self.circleDiameter = 50

    def objectif_couvert_animation(self):
        self.canvas.delete(self.animation)
        self.animation = None



class Mover(object):
    def __init__(self, canvas, wharehouse, position, objectif):
        lien_image = 'image/playerDown.png'
        self.image = tk.PhotoImage(file=lien_image)

        self.height = 64
        self.width = 64
        self.position = position
        self.canvas = canvas
        self.wharehouse = wharehouse
        self.objectif = objectif
        pos = self.position.de_position_a_coordonné_dans_canvas(self)
        self.canvas.create_image(pos.getX(), pos.getY(
        ), image=self.image, tags="movable", anchor=tk.NW)
        self.animation = None
        self.circleDiameter = 50

    def getHeight(self):
        return self.height

    def getWidth(self):
        return self.width

    def isMoveable(self):
        return True

    def deplace_dans_le_canvas(self, direction):
        if direction == Direction.Up:
            dir_x = 0
            dir_y = -1
        elif direction == Direction.Down:
            dir_x = 0
            dir_y = 1
        elif direction == Direction.Left:
            dir_x = -1
            dir_y = 0
        elif direction == Direction.Right:
            dir_x = 1
            dir_y = 0
        else:
            dir_x = 0
            dir_y = 0

        new_x = self.position.getX() * 64
        new_y = self.position.getY() * 64
        self.canvas.coords(self.image_id, new_x, new_y)


    def canMove(self, direction):
        new_position = self.position.position_suivante(direction, 1)

        if not new_position.est_valide_dans_entrepot(self.wharehouse):
            return False

        if not self.wharehouse.hasFreePlaceAt(new_position):
            return False

        if self.wharehouse.get_element(new_position) and self.wharehousget_element(new_position).isMovable():
            new_box_position = new_position.position_suivante(direction, 1)
            if not new_box_position.est_valide_dans_entrepot(self.wharehouse) or not self.wharehouse.hasFreePlaceAt(new_box_position):
                return False

        return True


    def se_deplace_vers(self, direction):
        if self.canMove(direction):
            self.position = self.position.position_suivante(direction, 1)

            self.deplace_dans_le_canvas(direction)


    def image_relative_a_la_direction(self, dir):
        position = self.position.de_position_a_coordonné_dans_canvas(self)

        if dir == Direction.Up:
            lien_image = 'image/playerUp.png'
            self.image = tk.PhotoImage(file=lien_image)
            self.canvas.create_image(position.getX(), position.getY(
            ), image=self.image, tags="movable", anchor=tk.NW)
        elif dir == Direction.Down:
            lien_image = 'image/playerDown.png'
            self.image = tk.PhotoImage(file=lien_image)
            self.canvas.create_image(position.getX(), position.getY(
            ), image=self.image, tags="movable", anchor=tk.NW)
        elif dir == Direction.Left:
            lien_image = 'image/playerLeft.png'
            self.image = tk.PhotoImage(file=lien_image)
            self.canvas.create_image(position.getX(), position.getY(
            ), image=self.image, tags="movable", anchor=tk.NW)
        elif dir == Direction.Right:
            lien_image = 'image/playerRight.png'
            self.image = tk.PhotoImage(file=lien_image)
            self.canvas.create_image(position.getX(), position.getY(
            ), image=self.image, tags="movable", anchor=tk.NW)


    def push(self, direction):
        self.image_relative_a_la_direction(direction)
        if not self.canMove(direction):
            self.active_animation_incapable_pousser()
            return
        self.se_deplace_vers(direction)

    def xsbChar(self):
        if self.under.isFreePlace():
            return '@'
        else:
            return '+'

    def isFreePlace(self):
        return False

    def active_animation_incapable_pousser(self):
        self.canvas.after(1, self.impossiblePushAnimation)

    def delete_old_animation(self):
        self.canvas.delete(self.animation)
        self.animation = None
        self.circleDiameter = 50

    def impossiblePushAnimation(self):
        if self.circleDiameter < 0:
            self.delete_old_animation()
            return

        if isinstance(self.animation, list):  
            for line in self.animation:
                self.canvas.delete(line)
        else:
            self.canvas.delete(self.animation)

        x_start = self.position.getX() * 64
        y_start = self.position.getY() * 64
        x_end = x_start + self.circleDiameter
        y_end = y_start + self.circleDiameter

        self.animation = [
            self.canvas.create_line(x_start, y_start, x_end, y_end, fill="purple", width=2),
            self.canvas.create_line(x_start, y_end, x_end, y_start, fill="purple", width=2),
        ]

        self.circleDiameter -= 0.5

        self.canvas.after(1, self.impossiblePushAnimation)


class Game(object):
    def __init__(self, root, xsbMatrix, instance_socoban, instance_menu):
        self.root = root
        self.wharehouse = WharehousePlan()
        self.won = False
        self.instance_socoban = instance_socoban
        self.instance_menu = instance_menu

        self.nbrows = len(xsbMatrix)
        self.nbcolumns = 0

        for line in xsbMatrix:
            nbc = len(line)
            if nbc > self.nbcolumns:
                self.nbcolumns = nbc

        self.height = self.nbrows * 64
        self.width = self.nbcolumns * 64

        self.canvas = tk.Canvas(
            self.root, width=self.width, height=self.height, bg="gray")
        self.canvas.pack()

        self.xsbMatrix = xsbMatrix
        self.Xsb_to_Wharehouse(
            xsbMatrix, self.nbrows, self.nbcolumns, self.wharehouse)
        self.keypressed_counter = 0

        self.key_counter_label = tk.Label(
            self.root, text="Key Presses: 0", font=("Helvetica", 16))
        self.key_counter_label.pack(pady=10)  
        self.root.bind("<Key>", self.keypressed)

   # convertit la matrice xsb en une matrice  
    def Xsb_to_Wharehouse(self, xsbMatrix, nbrows, nbcolumns, wharehouse):
        #   '#' = wall, '$' = box, '.' = goal, '*' = box on goal, '@' = mover, '+' = mover on goal,
        #   '-' = floor, ' ' = floor
        self.staticMatrix = []
        self.movableMatrix = []
        for lineIdx in range(nbrows):
            self.staticMatrix.append([])
            self.movableMatrix.append([])
            for elemIdx in range(nbcolumns):
                self.staticMatrix[lineIdx].append(None)
                self.movableMatrix[lineIdx].append(None)

        y = 0
        for lineIdx in range(len(xsbMatrix)):
            x = 0
            for elemIdx in range(len(xsbMatrix[lineIdx])):
                e = xsbMatrix[lineIdx][elemIdx]
                if e == '#':
                    wall = Wall(self.canvas, Position(elemIdx, lineIdx))
                    self.staticMatrix[y][x] = wall
                    self.wharehouse.place_element(
                        Position(elemIdx, lineIdx), wall)
                elif e == '@':
                    mover = Mover(self.canvas, self.wharehouse,
                                  Position(elemIdx, lineIdx), objectif=False)
                    self.mover = mover
                    self.movableMatrix[y][x] = mover
                    self.wharehouse.place_element(
                        Position(elemIdx, lineIdx), mover)
                    self.moverX = x
                    self.moverY = y
                elif e == '$':
                    box = Box(self.canvas, self.wharehouse,
                              Position(elemIdx, lineIdx), objectif=False)
                    self.movableMatrix[y][x] = box
                    self.wharehouse.place_element(
                        Position(elemIdx, lineIdx), box)
                elif e == '.':
                    goal = Goal(self.canvas, Position(elemIdx, lineIdx))
                    self.staticMatrix[y][x] = goal
                    self.wharehouse.place_element(
                        Position(elemIdx, lineIdx), goal)
                elif e == '*':
                    box_on_goal = Box(self.canvas, self.wharehouse, Position(
                        elemIdx, lineIdx), objectif=True)
                    goal = Goal(self.canvas, Position(elemIdx, lineIdx))
                    self.movableMatrix[y][x] = box_on_goal
                    self.staticMatrix[y][x] = goal
                    self.wharehouse.place_element(
                        Position(elemIdx, lineIdx), box_on_goal)
                else:
                    self.staticMatrix[y][x] = None
                    self.movableMatrix[y][x] = None
                x += 1
            y += 1
            x = 0

        self.canvas.tag_raise("movable", "static")
        
    #modifie la position du joueu en fonction des touches pressé appelle a fonction pour verifié la victoire
    def keypressed(self, event):
        self.keypressed_counter += 1
        self.key_counter_label.config(
            text=f"Key Presses: {self.keypressed_counter}")

        x, y = self.moverX, self.moverY
        direction = None
        if event.keysym == 'Up':
            direction = Direction.Up
        elif event.keysym == 'Down':
            direction = Direction.Down
        elif event.keysym == 'Left':
            direction = Direction.Left
        elif event.keysym == 'Right':
            direction = Direction.Right

        newpos = self.mover.position.position_suivante(direction, 1)
        posY, posX = newpos.getY(), newpos.getX()

        if isinstance(self.staticMatrix[posY][posX], Wall) == False:
            self.moveMover(x, y, newpos, direction)
        else:
            self.moverX = x
            self.moverY = y
            self.canvas.delete(self.mover)
            self.mover.image_relative_a_la_direction(direction)
            self.mover.active_animation_incapable_pousser()

        self.verifie_victoire()


    def moveMover(self, x, y, new_pos, direction):
        posY, posX = new_pos.getY(), new_pos.getX()
        self.movableMatrix[y][x] = None

        if (isinstance(self.movableMatrix[posY][posX], Box)):

            new_box_position = new_pos.position_suivante(direction, 1)
            newBoxX, newBoxY = new_box_position.getX(), new_box_position.getY()

            if (isinstance(self.staticMatrix[newBoxY][newBoxX], Wall) or isinstance(self.movableMatrix[newBoxY][newBoxX], Box)):
                self.mover.active_animation_incapable_pousser()
                return

            if (isinstance(self.staticMatrix[newBoxY][newBoxX], Goal)):
                box = Box(self.canvas, self.wharehouse,
                          new_box_position, objectif=True)

            else:
                box = Box(self.canvas, self.wharehouse,
                          new_box_position, objectif=False)

            self.movableMatrix[y][x] = None
            self.movableMatrix[newBoxY][newBoxX] = box
            self.movableMatrix[posY][posX].se_deplace_vers(direction)

        self.mover.position = new_pos
        self.moverX, self.moverY = self.mover.position.getX(), self.mover.position.getY()
        self.mover.se_deplace_vers(direction)
        self.canvas.delete(self.mover)
        self.mover.image_relative_a_la_direction(direction)
        self.canvas.tag_raise("movable", "static")
        
    # verifie la victoire et affiche les statistique du joueur durant à la fin de la partie
    def verifie_victoire(self):
        toutes_les_boites_sont_sur_objectifs = all(
            isinstance(element, Box) and element.on_goal
            for ligne in self.movableMatrix
            for element in ligne if isinstance(element, Box)
        )

        if toutes_les_boites_sont_sur_objectifs:
            self.won = True
            tkMessageBox.showinfo("Victoire!", "Félicitations, vous avez terminé le niveau!")

            score = 10000 - self.keypressed_counter
            self.instance_socoban.gestion_scores.mettre_a_jour_score(
                self.instance_socoban.player_name, score
            )

            continuer = tkMessageBox.askyesno(
                "Continuer ?", "Voulez-vous passer au niveau suivant ?"
            )

            if continuer:
                self.instance_socoban.restart_game()
            else:
                self.instance_socoban.Resultat()
                tkMessageBox.showinfo("Fin", "Merci d'avoir joué !")
                self.instance_socoban.root.destroy()  



class GestionScores:
    def __init__(self, fichier_scores="scores.json"):
        self.fichier_scores = fichier_scores
        #initialise une variable avec un dictionnaire des score présent dans le fichier
        self.scores = self.charger_scores()

    #retourne un dictionnaire des score présent dans le fichier
    def charger_scores(self):
        try:
            with open(self.fichier_scores, "r") as f:
                return json.load(f)
        except FileNotFoundError:
            return {"joueurs": []}  

    def sauvegarder_scores(self):
        with open(self.fichier_scores, "w") as f:
            json.dump(self.scores, f, indent=4)  

    def obtenir_joueurs(self):
        return self.scores["joueurs"]

    def ajouter_joueur(self, nom):
        if not any(joueur["nom"] == nom for joueur in self.scores["joueurs"]):
            self.scores["joueurs"].append({"nom": nom, "score": 0})
            self.sauvegarder_scores()
    # permet de modifier le score d'un joueur si il était déja dans la base de donné 
    def mettre_a_jour_score(self, nom, nouveau_score):
        joueur_trouve = False
        for joueur in self.scores:
            if joueur["joueur"] == nom:  
                joueur["score"] = max(joueur["score"], nouveau_score)  
                joueur_trouve = True
                break
        if not joueur_trouve:
            self.scores.append({"joueur": nom, "score": nouveau_score})
        self.sauvegarder_scores()  
        
class Sokoban(object):
    def __init__(self):
        # Initialisation de la gestion des scores
        self.gestion_scores = GestionScores()  

        # Variables pour le pseudo du joueur et la fenêtre principale
        self.player_name = None
        self.root = tk.Tk()
        self.root.title("Menu") 
        self.root.geometry("800x600") 

        # Chargement de l'image de fond pour le menu principal
        self.bg_image = tk.PhotoImage(file="image/Background.png")
        bg_label = tk.Label(self.root, image=self.bg_image)
        bg_label.place(relwidth=1, relheight=1) 

        # Définition des polices rétro pour l'affichage
        retro_font = ("Courier", 16, "bold")
        retro_font_large = ("Courier", 24, "bold")

        # Titre du jeu dans le menu principal
        title = tk.Label(self.root, text="SOKOBAN",
                         font=retro_font_large, fg="#00FF00", bg="#000000")
        title.pack(pady=20)

        # Champ pour entrer le niveau à jouer
        tk.Label(self.root, text="Niveau :", font=retro_font,
                 fg="#FFFFFF", bg="#000000").pack(pady=10)
        self.level_entry = tk.Entry(
            self.root, font=retro_font, justify="center")
        self.level_entry.pack(pady=10)

        # Champ pour entrer le pseudo du joueur
        tk.Label(self.root, text="Pseudo :", font=retro_font,
                 fg="#FFFFFF", bg="#000000").pack(pady=10)
        self.level_player = tk.Entry(
            self.root, font=retro_font, justify="center")
        self.level_player.pack(pady=10)

        # Bouton pour commencer une nouvelle partie
        start_button = tk.Button(self.root, text="Nouvelle Partie", font=retro_font,
                                 fg="#FFFFFF", bg="#FF4500", command=self.start_game)
        start_button.pack(pady=20)

        # Variables pour le niveau sélectionné et la gestion du jeu
        self.selected_level = 0
        self.player_name = None
        self.level = None

    def start_game(self):
        # Démarre une nouvelle partie en vérifiant les entrées utilisateur
        try:
            # Récupération du niveau sélectionné (doit être un entier)
            self.selected_level = int(self.level_entry.get())
        except ValueError:
            tkMessageBox.showinfo(
                "Erreur", "Veuillez entrer un niveau valide (nombre entier).")
            return

        # Récupération du pseudo du joueur
        self.player_name = self.level_player.get()
        if not self.player_name.strip():
            tkMessageBox.showinfo("Erreur", "Veuillez entrer un pseudo.")
            return

        # Vérifie si le niveau sélectionné existe
        if 0 <= self.selected_level < len(SokobanXSBLevels):
            self.root.destroy() 
            self.launch_level() 
        else:
            tkMessageBox.showinfo("Erreur", "Le niveau n'est pas valide.")

    def restart_game(self):
        # Passe au niveau suivant
        self.selected_level += 1
        if self.selected_level < len(SokobanXSBLevels):
            # Détruit le canvas actuel et initialise le nouveau niveau
            self.level.canvas.destroy()
            self.level = Game(
                self.root, SokobanXSBLevels[self.selected_level], self, self
            )
        else:
            # Fin du jeu si tous les niveaux sont terminés
            tkMessageBox.showinfo("Fin", "Félicitations, vous avez terminé tous les niveaux !")
            self.root.destroy()

    def launch_level(self):
        # Lance un niveau du jeu
        self.root = tk.Tk()  
        self.root.resizable(True, True)  
        self.root.title("Sokoban") 
        self.level = Game(self.root, SokobanXSBLevels[self.selected_level], self, self)

        # Initialisation du chronomètre pour le suivi du temps
        self.start_time = 0
        self.running = True
        self.chronometer_label = tk.Label(
            self.root, text="Time: 00:00", font=("Helvetica", 16))
        self.chronometer_label.pack(pady=10)

        # Démarrage du chronomètre
        self.chronometre()
        self.play()

    def chronometre(self):
        # Met à jour le chronomètre toutes les secondes
        if self.running:
            self.start_time += 1
            self.elapsed_time = timedelta(seconds=self.start_time)
            self.chronometer_label.config(text=f"Time: {self.elapsed_time}")
            self.timer_id = self.root.after(1000, self.chronometre)

        if self.level.won:
            # Arrête le chronomètre si le niveau est terminé
            self.running = False
            self.root.after_cancel(self.timer_id)

    def Resultat(self):
        # Affiche les statistiques à la fin d'un niveau
        vitesse = self.level.keypressed_counter * \
            100 // self.elapsed_time.total_seconds()  # Calcul de la vitesse
        tkMessageBox.showinfo("Statistiques", f"Vitesse : {vitesse} (pas/s * 100)")
        tkMessageBox.showinfo("Pseudo", f"Joueur : {self.player_name}")

    def play(self):
        self.root.mainloop()

jeu = Sokoban()
jeu.play()
